# VariantSpark-k Medium Parquet
 - 5,000 samples  
 - 100,000,000 variables   
 - ?? minutes to run
 
 Realistic production-sized job (217 GB) - takes < ?? minutes to run w/ 4 qty of r4.4xlarge EC2 instance & Kubernetes
 
 NOTE: Two trial runs have completed in the notebook, but NOT returned '0', also the pods had to be stopped manually and did NOT return any results.

### Parameters
 - input bucket location - CSIRO S3 bucket (Sydney)
 - input files `data_s5000_v100000000...` and fc `resp`
 - number of Spark executors 56 w/ 7 GB RAM each
 - number of RF trees 500, w/ batch of 28
 - file type is `-it parquet`
 - default `mtry`
 - calculate OOB as `-ro` paramater is configured

In [1]:
%%bash

set -e

MASTER=https://kubernetes.default.svc:443
INPUT_BUCKET=variant-spark

function fatal_error () {
	echo "ERROR: $1" 1>&2
	exit 1
}

if [ -z ${MASTER+x} ];
    then
        echo "You must set the MASTER environment variable to a kubernetes API endpoint";
        echo "Example: https://ABC.sk1.us-west-2.eks.amazonaws.com:443"
        exit 1
fi

if [ -z ${INPUT_BUCKET+x} ];
    then
        echo "You must set the INPUT_BUCKET environment variable to a bucket containing input data";
        echo "Example: variant-spark-k-storage"
        exit 1
fi

[[ $(type -P "spark-submit") ]] || fatal_error  "\`spark-submit\` cannot be found. Please make sure it's on your PATH."

spark-submit \
    --class au.csiro.variantspark.cli.VariantSparkApp \
    --driver-class-path ./conf \
    --master k8s://${MASTER} \
    --deploy-mode cluster \
    --name VariantSparkMediumParquet \
    --conf spark.kubernetes.authenticate.driver.serviceAccountName=spark \
    --conf spark.executor.instances=120 \
    --conf spark.executor.memory=7g \
    --conf spark.kubernetes.container.image=jamesrcounts/variantspark:002 \
    --jars http://central.maven.org/maven2/org/apache/hadoop/hadoop-aws/2.7.3/hadoop-aws-2.7.3.jar,http://central.maven.org/maven2/com/amazonaws/aws-java-sdk/1.7.4/aws-java-sdk-1.7.4.jar,http://central.maven.org/maven2/joda-time/joda-time/2.9.9/joda-time-2.9.9.jar \
    local:///opt/spark/jars/variant-spark_2.11-0.2.0-SNAPSHOT-all.jar importance \
        -if s3a://${INPUT_BUCKET}/datasets/synthetic/data_s5000_v100000000_r13.parquet \
        -ff s3a://${INPUT_BUCKET}/datasets/synthetic/labels_s5000_v100000000_r13_f0.125.csv \
        -fc resp \
        -it parquet \
        -v \
        -rn 200 \
        -rbs 28 \
        -ro "$@"


2018-09-16 08:42:26 WARN  DriverServiceBootstrapStep:66 - Driver's hostname would preferably be variantsparkmediumparquet-58d581b28f653df791a7ce79e21d9628-driver-svc, but this is too long (must be <= 63 characters). Falling back to use spark-1537087346159-driver-svc as the driver service's name.
2018-09-16 08:42:27 INFO  LoggingPodStatusWatcherImpl:54 - State changed, new state: 
	 pod name: variantsparkmediumparquet-58d581b28f653df791a7ce79e21d9628-driver
	 namespace: default
	 labels: spark-app-selector -> spark-d0fa92b7b49d44799f3cfb1e2e82e5f1, spark-role -> driver
	 pod uid: 707fffa6-b98c-11e8-8a8e-0a0cd35f5fb0
	 creation time: 2018-09-16T08:42:26Z
	 service account name: spark
	 volumes: spark-init-properties, download-jars-volume, download-files-volume, spark-token-l2lzk
	 node name: N/A
	 start time: N/A
	 container images: N/A
	 phase: Pending
	 status: []
2018-09-16 08:42:27 INFO  LoggingPodStatusWatcherImpl:54 - State changed, new state: 
	 pod name: variantsparkmediumparquet